In [1]:
import csv
import re
import os
import mmap
import pprint
from pprint import pprint
import pandas as pd
import numpy as np
!pip install gensim -U
import gensim
from gensim.parsing.preprocessing import remove_stopwords
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Dataset: https://www.kaggle.com/datasets/oswinrh/bible?resource=download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.8 MB 27.4 MB/s 
     |████████████████████████████████| 1.3 MB 61.7 MB/s 
     |████████████████████████████████| 182 kB 76.0 MB/s 
     |████████████████████████████████| 7.6 MB 46.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cbaee4334f284cfd6bf5aa0ee526b2a52b82525c92dbc03bffd418270dca3473
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully bui

In [2]:
def read_50_as_batch(file_handler, chunk_size=50):
    chunk = []
    for line in file_handler:
        chunk.append(line)
        if len(chunk) == chunk_size:
            yield chunk
            chunk = []

    # Yielding last chunk
    if chunk:
        yield chunk

# Empty text file before writing to it
open(f'filteredtext.csv', 'w').close()

with open('./t_kjv.csv') as file_handler:
  # Write chunk into file
  for chunk in read_50_as_batch(file_handler):
    for row in chunk:
      row = row.lower()
      # Since Gensim won't remove stopwords that have punctuations directly in front or affixed to the stopwords, need to create space before and after these symbols:
      row = row.replace('"', ' " ')
      row = row.replace(',', ' , ')
      row = row.replace('.', ' . ')
      row = row.replace(':', ' : ')
      row = row.replace(';', ' ; ')
      row = row.replace('?', ' ? ')
      row = row.replace(",", "|", 4)
      #row = re.sub(",,,,", "|", row, 0, re.IGNORECASE | re.VERBOSE | re.MULTILINE)
      #print(row)
      filtered_sentence = remove_stopwords(row)

      # After Gensim has removed stopwords, remove the punctuations in the sentences
      filtered_sentence = filtered_sentence.replace(' , ', ' ')
      filtered_sentence = filtered_sentence.replace(' . ', ' ')
      filtered_sentence = filtered_sentence.replace(' : ', ' ')
      filtered_sentence = filtered_sentence.replace(' ; ', ' ')
      filtered_sentence = filtered_sentence.replace(' ? ', ' ')
      filtered_sentence = filtered_sentence.replace(' | ', '|')

      #print(filtered_sentence)

      # Write filtered sentence to a csv file
      f=open('filteredtext.csv','a')
      writer = csv.writer(f)
      csvdata = filtered_sentence.split("|")
      writer.writerow([csvdata[0], csvdata[1], csvdata[2], csvdata[3], csvdata[4]])

In [3]:
# Empty text file before writing to it
open('cleaned.csv', 'w').close()

headercounter = 0

for chunk in pd.read_csv('filteredtext.csv', chunksize=25):
  # Remove double quotes in text field
  chunk['t'] = chunk['t'].apply(lambda x: str(x).replace('"', ""))
  # Remove period in text field
  chunk['t'] = chunk['t'].apply(lambda x: str(x).replace('.', ""))
  # Remove leading space
  chunk['t'] = chunk['t'].apply(lambda x: str(x).lstrip())

  # Only add header for the first chunk
  if(headercounter == 0):
    headerList = ['id', 'b', 'c', 'v', 't']
    chunk.to_csv("cleaned.csv", mode='a', header=headerList)
  # Don't add header after the first chunk
  else:
    chunk.to_csv("cleaned.csv", mode='a', header=None)
  headercounter+=1


In [4]:
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# How to speed this up:
# https://stackoverflow.com/questions/61588381/speed-up-embedding-of-2m-sentences-with-roberta

def read_csv_and_embed(file, file_embed, skiprows=None):
  # Empty text file before writing to it
  open(file_embed, 'w').close()

  sentence_and_embeddings_list = []

  with open(file) as f:
    if skiprows:
      f.readlines(skiprows)
    reader = pd.read_csv(f, chunksize=1) 
    for chunk in reader:
      # Encode verse text into embeddings
      embeddings = model.encode(str(chunk['t']))
      sentence_and_embeddings_list.append(embeddings)

  # Save the verse text embeddings into a file
  with open(file_embed,'wb') as fe:
    np.save(fe, sentence_and_embeddings_list)

embed_file_path = './bibleverses2.embed'

# Check if embedding file exists, if yes, no need to call the read_csv_and_embed function again
if os.path.exists(embed_file_path) and os.path.getsize(embed_file_path) > 0:
  print(f"{embed_file_path} exists and not empty")

# If embedding file doesn't exist, run embedding function
else:
  read_csv_and_embed('cleaned.csv', './bibleverses2.embed')

In [42]:
# Books in the Bible
books_dictionary = {}

# Adding to books_dictionary a key value pair of index, and its real book name
books_dictionary.update({1 : "Genesis"})
books_dictionary.update({2 : "Exodus"})
books_dictionary.update({3 : "Leviticus"})
books_dictionary.update({4 : "Numbers"})
books_dictionary.update({5 : "Deuteronomy"})
books_dictionary.update({6 : "Joshua"})
books_dictionary.update({7 : "Judges"})
books_dictionary.update({8 : "Ruth"})
books_dictionary.update({9 : "1 Samuel"})
books_dictionary.update({10 : "2 Samuel"})
books_dictionary.update({11 : "1 Kings"})
books_dictionary.update({12 : "2 Kings"})
books_dictionary.update({13 : "1 Chronicles"})
books_dictionary.update({14 : "2 Chronicles"})
books_dictionary.update({15 : "Ezra"})
books_dictionary.update({16 : "Nehemiah"})
books_dictionary.update({17 : "Esther"})
books_dictionary.update({18 : "Job"})
books_dictionary.update({19 : "Psalms"})
books_dictionary.update({20 : "Proverbs"})
books_dictionary.update({21 : "Ecclesiastes"})
books_dictionary.update({22 : "Song of Solomon"})
books_dictionary.update({23 : "Isaiah"})
books_dictionary.update({24 : "Jeremiah"})
books_dictionary.update({25 : "Lamentations"})
books_dictionary.update({26 : "Ezekiel"})
books_dictionary.update({27 : "Daniel"})
books_dictionary.update({28 : "Hosea"})
books_dictionary.update({29 : "Joel"})
books_dictionary.update({30 : "Amos"})
books_dictionary.update({31 : "Obadiah"})
books_dictionary.update({32 : "Jonah"})
books_dictionary.update({33 : "Micah"})
books_dictionary.update({34 : "Nahum"})
books_dictionary.update({35 : "Habakkuk"})
books_dictionary.update({36 : "Zephaniah"})
books_dictionary.update({37 : "Haggai"})
books_dictionary.update({38 : "Zechariah"})
books_dictionary.update({39 : "Malachi"})
books_dictionary.update({40 : "Matthew"})
books_dictionary.update({41 : "Mark"})
books_dictionary.update({42 : "Luke"})
books_dictionary.update({43 : "John"})
books_dictionary.update({44 : "Acts"})
books_dictionary.update({45 : "Romans"})
books_dictionary.update({46 : "1 Corinthians"})
books_dictionary.update({47 : "2 Corinthians"})
books_dictionary.update({48 : "Galatians"})
books_dictionary.update({49 : "Ephesians"})
books_dictionary.update({50 : "Philippians"})
books_dictionary.update({51 : "Colossians"})
books_dictionary.update({52 : "1 Thessalonians"})
books_dictionary.update({53 : "2 Thessalonians"})
books_dictionary.update({54 : "1 Timothy"})
books_dictionary.update({55 : "2 Timothy"})
books_dictionary.update({56 : "Titus"})
books_dictionary.update({57 : "Philemon"})
books_dictionary.update({58 : "Hebrews"})
books_dictionary.update({59 : "James"})
books_dictionary.update({60 : "1 Peter"})
books_dictionary.update({61 : "2 Peter"})
books_dictionary.update({62 : "1 John"})
books_dictionary.update({63 : "2 John"})
books_dictionary.update({64 : "3 John"})
books_dictionary.update({65 : "Jude"})
books_dictionary.update({66 : "Revelation"})

# Bible search method which will return 3 best hit bible verses, taking in a query as input
def bible_search(input):
  embed_file_path = 'bibleverses2.embed'

  input = model.encode(input)

  with open(embed_file_path, 'rb') as f:
    embeddings = np.load(f)

  # List of Cosine Similarity Values
  cosine_values = []
  
  # Append cosine similairty values in list
  for i in range(0, len(embeddings)):
    cosine_values.append(cosine_similarity(input.reshape(1,-1), embeddings[i].reshape(-1, 768)))

  # Look up top bible verses in t_kjv.csv:
  df = pd.read_csv('t_kjv.csv')
  index_of_verse1 = cosine_values.index(max(cosine_values))
  verse1 = df.iloc[[index_of_verse1]]

  cosine_values_float = []
  for item in cosine_values:
    cosine_values_float.append(item)

  cosine_values_float.sort(reverse = True)

  index_of_verse2 = cosine_values.index(cosine_values_float[1].astype(np.float32))
  index_of_verse3 = cosine_values.index(cosine_values_float[2].astype(np.float32))

  verse2 = df.iloc[[index_of_verse2]]
  verse3 = df.iloc[[index_of_verse3]]

  return verse1, verse2, verse3

# Passing query string into the bible_search method
bibleverse1, bibleverse2, bibleverse3 = bible_search('How to be a good neighbor?')

# Print 3 most relevant bible verses in relation to the query string
print(f"{books_dictionary.get(bibleverse1['b'].item())}\nChapter {bibleverse1['c'].item()} Verse {bibleverse1['v'].item()}\n{bibleverse1['t'].item()}\n")
print(f"{books_dictionary.get(bibleverse2['b'].item())}\nChapter {bibleverse2['c'].item()} Verse {bibleverse2['v'].item()}\n{bibleverse2['t'].item()}\n")
print(f"{books_dictionary.get(bibleverse3['b'].item())}\nChapter {bibleverse3['c'].item()} Verse {bibleverse3['v'].item()}\n{bibleverse3['t'].item()}\n")



Romans
Chapter 15 Verse 1
We then that are strong ought to bear the infirmities of the weak, and not to please ourselves.

Isaiah
Chapter 41 Verse 5
The isles saw it, and feared; the ends of the earth were afraid, drew near, and came.

Exodus
Chapter 20 Verse 16
Thou shalt not bear false witness against thy neighbor.

